# Chiral solutions for Type-II Dirac seesaw with a Dark Symmetry
![img](./schema.svg)

In [2]:
import pandas as pd
import numpy as np
import itertools

In [182]:
pd.set_option('display.max_colwidth',500)

### Load full solutions

In [3]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

Number of solutions with repeated charges: $(\nu,\nu,\cdots)$

In [4]:
ds

,l,k,solution,gcd,n
0,"[-1, 0, -1, 0, -1]","[-1, 0, -1, 1, -1]","[1, 4, 4, 4, -5, -5, -5, -5, -5, 6, 6]",1,11
1,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7
2,"[1, -1, 0, 1, -1]","[-2, 0, -1, -2, 0]","[1, 1, -2, -2, -3, -3, 5, 5, 5, 5, -6, -6]",2,12
3,"[-1, 1, -1, 1, -1]","[-2, -1, 0, -1, 0]","[1, 1, 1, 1, 2, 2, 2, -3, -4, -4, -6, 7]",1,12
4,"[1, 0, 1, 0, -1]","[-2, -1, -2, -1, 0]","[1, 1, 1, -2, -2, -5, -5, 6, 6, 7, -8]",1,11
...,...,...,...,...,...
390069,"[-3, -15, -6]","[-3, 8, 9]","[8, -9, -9, 11, 25, -29, -29, 32]",36,8
390070,"[-10, -5, -15]","[-10, -12, 12]","[5, 5, 5, -17, -27, -27, 28, 28]",100,8
390071,"[15, -1, 15]","[-14, -13, -14]","[14, -15, -16, 17, -30, 31, 31, -32]",1,8
390072,"[-15, -9, -1]","[-13, -14, -12]","[11, -13, -15, 19, 23, -26, -30, 31]",18,8


## Filter two set of repeated charges
with one of the repeated 3 times
$$(\nu,\nu[,\nu],m,m,m,\psi_1,\psi_2,\cdots)$$


In [238]:
dsp=ds[ds['solution'].apply(lambda l: len(l)-len(set(l))>2)].reset_index(drop=True)
dsp.shape

(48290, 5)

In [239]:
dsp=dsp[dsp['solution'].apply(lambda l: 3 in [ y for y in [l.count(x) for x in set(l) if l.count(x)!=1] 
                                  if y==2 or y==3
                                ])].reset_index(drop=True)

In [240]:
dsp['solution']

0           [1, 4, 4, 4, -5, -5, -5, -5, -5, 6, 6]
1                         [1, 2, 2, -3, -3, -3, 4]
2         [1, 1, 1, 1, 2, 2, 2, -3, -4, -4, -6, 7]
3           [1, 1, 1, -2, -2, -5, -5, 6, 6, 7, -8]
4        [1, 1, 1, -2, 6, 6, 6, -7, -7, -7, -7, 9]
                           ...                    
21462          [3, 3, 3, 4, -14, 26, -27, -27, 29]
21463                [9, 9, -24, -24, -24, 25, 29]
21464         [13, -17, -17, -17, 20, 20, 23, -25]
21465         [13, 13, -15, -15, -15, 20, 27, -28]
21466             [5, 5, 5, -17, -27, -27, 28, 28]
Name: solution, Length: 21467, dtype: object

Some reordering

In [241]:
dsp['nmax']=dsp['solution'].apply(lambda l: map(abs,l)).apply(max)
dsp=dsp.sort_values(['n','nmax']).reset_index(drop=True)
#Show the first and last one solutions
dsp.iloc[[0,1,2,-1]]

,l,k,solution,gcd,n,nmax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5
1,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,4
2,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7,8
21466,"[-2, -3, -2, -3, -2]","[-1, 3, -1, 3, -1]","[14, -18, -24, -24, -24, 27, 27, 28, 29, 29, -32, -32]",2,12,32


## 1. Check phenomenological conditions
$$(\nu,\nu[,\nu],m,m,m,\psi_1,\psi_2,\cdots)$$


$$ \nu+m+s=0$$

$$ \psi_i+\psi_j=|s|$$

Test:

In [242]:
s=-1
ms=set([x for x in itertools.permutations([-4, -4, 5],2) if x[0]<=x[1]])
xs=[ x for x in ms if abs(x[0]+x[1])==abs(s) ]
[x for sublist in xs for x in sublist]

[-4, 5]

Real implementation

In [243]:
def get_massles(l,s):
    if len(l)==0:
        return []
    elif len(l)==1:
        mssls=[(l[0],l[0])]
    elif len(l)==2:
        mssls=[tuple(l)]
    else:
        mssls=set([x for x in itertools.permutations(l,2) if x[0]<=x[1] ])
    xs=[ x for x in mssls if abs(x[0]+x[1])==abs(s) ]
    #flatten list of list into list
    massive=[x for sublist in xs for x in sublist]

    return list( set(l).difference(set(massive)) )    

In [244]:
def get_sp(massless):
    """
    For a list of numbers `x_i` find the best `sp` value such that
    all the elements of the list satisfy:
       |sp|=|x_i + x_j|
    * If the list is already empty return None
    * If not found `sp` returns 0
    """
    if len(massless)==0:
        return None
    elif len(massless)==1:
        drs=[(massless[0],massless[0])]
    elif len(massless)==2:
        drs=[tuple(massless)]
    else:
        drs=[x for x in itertools.permutations(massless,2) if x[0]<=x[1]]
    sps=set(abs(sum(x)) for x in drs)    
    for sp in sps:
        msv =([dr for dr in drs if abs(sum(dr))==sp])
        massive=set([x for sublist in msv for x in sublist])
        msl=([dr for dr in drs if abs(sum(dr))!=sp])
        massless=set( [x for sublist in msl for x in sublist])
        majorana=massless.difference(massive)
        majorana=[m for m in majorana if abs(2*m)!=sp]
        if not majorana:
            return sp
        else:
            return 0        

In [245]:
Type='TII-X'
#l=[1,-9,-9,-9,13,18,18,18,-19,-22]
#def conditions(l):
def condition(l,Type='TII-X'):
#if True:   
    sls=[]
    ms=set([x for x in l if l.count(x)==3])
    νs=set([x for x in l if l.count(x)==2 or l.count(x)==3])
    mνs=[(m,ν) for m in ms for ν in νs if m!=ν]
    sold=0
    for mν in mνs:
        m=mν[0]
        ν=mν[1]
        s=-m-ν
        if abs(s)==abs(sold):
            continue
        else:
            sold=s            
        ψs=[x for x in l if x!=ν and x!=m]
        msslss=get_massles(ψs,s)
        sp=get_sp(msslss)
        d={'Type':Type,'m':m,'ν':ν,'s':s,'massless':msslss,'sp':sp}
        sls.append(d)
    #sp=None    
    solutions=[d for d in sls if d.get('massless')==[]]
    if not solutions:
        solutions=[d for d in sls if d.get('sp')]    
    return solutions

l=[1, 2, -6, -6, -6, 8, 9, 9, -11]
assert condition(l)[0].get('massless')==[]
l=[1, 1, 1, -4, -4, 5]
assert condition(l)[0].get('sp')==10
l=[1,-9,-9,-9,13,18,18,18,-19,-22]
assert condition(l)[0].get('sp')==18

In [246]:
dsp['tree-level']=dsp['solution'].apply(condition)

In [247]:
dsp=dsp[dsp['tree-level'].apply(len)>0].reset_index(drop=True)

In [248]:
dsp[dsp['tree-level'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0]

,l,k,solution,gcd,n,nmax,tree-level
4,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,8,9,"[{'Type': 'TII-X', 'm': 3, 'ν': -7, 's': 4, 'massless': [], 'sp': None}]"
29,"[3, 0, -1, 1]","[-6, -5, 5, -1]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,9,6,"[{'Type': 'TII-X', 'm': 2, 'ν': -3, 's': 1, 'massless': [], 'sp': None}]"
37,"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,9,11,"[{'Type': 'TII-X', 'm': -6, 'ν': 9, 's': -3, 'massless': [], 'sp': None}]"
46,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'Type': 'TII-X', 'm': 6, 'ν': -9, 's': 3, 'massless': [], 'sp': None}]"
51,"[-2, -1, 0]","[-1, -4, -5, -4]","[1, -5, 8, 8, 8, -11, -12, -12, 15]",2,9,15,"[{'Type': 'TII-X', 'm': 8, 'ν': -12, 's': 4, 'massless': [], 'sp': None}]"
...,...,...,...,...,...,...,...
4674,"[1, -3, 1, 2, 1]","[-3, -1, -3, -1, -3]","[5, 9, -13, -13, -13, -15, -19, 21, 23, 23, 23, -31]",4,12,31,"[{'Type': 'TII-X', 'm': -13, 'ν': 23, 's': -10, 'massless': [], 'sp': None}]"
4701,"[-1, -2, -1, -2, -1]","[-2, 0, -2, 2, -2]","[1, -2, -15, 16, -28, -28, 29, 29, 29, -30, 31, -32]",1,12,32,"[{'Type': 'TII-X', 'm': 29, 'ν': -28, 's': -1, 'massless': [], 'sp': None}]"
4705,"[-2, 2, -2, 1, -2]","[-1, -2, -1, 2, -1]","[2, -4, -4, -4, -7, 11, -14, -23, 25, 25, 25, -32]",3,12,32,"[{'Type': 'TII-X', 'm': 25, 'ν': -4, 's': -21, 'massless': [], 'sp': None}]"
4706,"[-1, -2, -1, -2, 1]","[-1, -2, 2, -2, 2]","[13, 13, -16, -16, -20, -20, 23, 29, 29, 29, -32, -32]",2,12,32,"[{'Type': 'TII-X', 'm': 29, 'ν': -32, 's': 3, 'massless': [], 'sp': None}]"


In [260]:
#zn=dsp[dsp['tree-level'].apply(lambda l: any( [True for d in l if abs(d.get('s'))==6 or abs(d.get('s'))==10 ]))]

In [267]:
def unconditional_stability(row,ZN={6:(2,3),10:(2,5),15:(3,5)},DMs={},check_massless=True):
    '''
    According to appendix of arXiv:1911.05515, 
    generalized to any N
    
    '''
    if check_massless:
        sd=[d for d in row['tree-level'] if d.get('massless')==[]]
    else:
        sd=row['tree-level']
        
    for i in range(len(sd)):
        DMs={}
        d=sd[i]
        N=abs(d['s'])
        l=[n for n in row['solution'] if n not in [d['m'],d['ν']] ]
        #Check for non-hidden chiral fermions
        if 0 in np.array(l)%N:
            DMs={'None':tuple([])}
            continue
        DM=[x for x in itertools.permutations(l,2) 
               if x[0]<=x[1] and abs(sum(x))==N ]
        DM=DM+[(n,n) for n in l if abs(2*n)==N]
        #rank 0
        for dm in DM:
            if dm[0]!=dm[1] and l.count(dm[0])!=l.count(dm[1]) and abs(2*dm[0])!=N and abs(2*dm[0])!=N:
                DMs={'None':tuple([])}
        #missing massles → TODO: Check BUG
        if check_massless and np.setdiff1d(l,[item for sublist in DM for item in sublist]).size>0:
            DMs={'None':tuple([])}
        if not DMs.get('None')==():
            try:
                p,q=ZN[N]
            except KeyError:
                DMs={'None':tuple([])}
                continue
            ps=[n*p for n in range(1,N) if n*p<N]
            qs=[n*q for n in range(1,N) if n*q<N]
            ω=[ sorted([y%N for y in x]) for x in DM]
            DMF=['ψ' if np.setdiff1d(x,ps).size==0 else 'χ' 
                   if np.setdiff1d(x,qs).size==0 else 'None' for x in  ω ]
            DMs=dict(zip(DMF,DM))
            break
    return DMs

In [268]:
p=np.array( [2,3,5,7,11,13,17,19,23,29,31] )
q=p.copy()
zpq= [ pp*qq for pp in p for qq in q if pp<qq] 
Zpq=sorted( set( zpq ) )

In [269]:
pqs=[ (pp,qq) for pp in p for qq in q if pp<qq]
ZN=dict(zip(Zpq,pqs))

In [270]:
Zpq[:3]

[6, 10, 14]

In [276]:
dsp['DMs']=dsp.apply(lambda row: unconditional_stability(row,ZN)  ,axis='columns')

In [278]:
dsp[dsp['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())]

,l,k,solution,gcd,n,nmax,tree-level,DMs
1059,"[-2, 0, 1, 0, -3]","[-2, -1, 0, -1, 1]","[1, 3, 5, -6, -6, -6, 8, -9, 12, 12, -14]",2,11,14,"[{'Type': 'TII-X', 'm': -6, 'ν': 12, 's': -6, 'massless': [], 'sp': None}]","{'None': (1, 5), 'χ': (3, 3), 'ψ': (-14, 8)}"
1120,"[-2, 0, -1, 0, 3]","[-2, -1, 0, -1, 1]","[1, -2, 6, 6, 6, -7, 8, -9, -12, -12, 15]",2,11,15,"[{'Type': 'TII-X', 'm': 6, 'ν': -12, 's': 6, 'massless': [], 'sp': None}]","{'ψ': (-2, 8), 'None': (-7, 1), 'χ': (-9, 15)}"
1224,"[-3, 0, -2, 0]","[-2, -4, -2, 1, -1]","[1, 3, 3, 6, 6, 6, -7, -10, -12, -12, 16]",6,11,16,"[{'Type': 'TII-X', 'm': 6, 'ν': -12, 's': 6, 'massless': [], 'sp': None}]","{'χ': (3, 3), 'None': (-7, 1), 'ψ': (-10, 16)}"
1563,"[2, -1, 2, 0]","[-3, -2, -3, -2, -3]","[2, -6, -6, -6, -8, -11, 12, 12, 15, 17, -21]",6,11,21,"[{'Type': 'TII-X', 'm': -6, 'ν': 12, 's': -6, 'massless': [], 'sp': None}]","{'ψ': (-8, 2), 'None': (-11, 17), 'χ': (-21, 15)}"
3730,"[-3, 0, -3, -1, 3]","[-3, -1, -3, -1, 1]","[2, 6, 6, 6, -7, -16, -20, -20, -20, 21, 21, 21]",8,12,21,"[{'Type': 'TII-X', 'm': -20, 'ν': 6, 's': 14, 'massless': [], 'sp': None}, {'Type': 'TII-X', 'm': 6, 'ν': -20, 's': 14, 'massless': [], 'sp': None}]","{'χ': (-7, -7), 'ψ': (-16, 2)}"


In [200]:
ds[ds['solution'].astype(str).str.contains('\[1, 2, -6, -6, -6')]# ,−6,−6,−6,8,9,9,−11

,l,k,solution,gcd,n
9814,"[2, 0, -1, -2, -1]","[-3, -2, 2, 1, -1]","[1, 2, -6, -6, -6, -7, -8, 9, 11, 13, 16, -19]",2,12
52248,"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,9
69480,"[-2, 1, -2, 2, 3]","[-3, -2, -3, 0, -3]","[1, 2, -6, -6, -6, -6, 8, -11, 12, 15, 18, -21]",12,12
75783,"[-3, 2, -2, -1, -3]","[-3, -2, -1, 2, 0]","[1, 2, -6, -6, -6, 8, -9, -9, 15, 15, 16, -21]",10,12
84578,"[2, 3, -3, 1, -3]","[-3, -1, -3, -1, 3]","[1, 2, -6, -6, -6, -7, 9, 12, -14, 18, 21, -24]",11,12
97647,"[-2, 0, 2, -3, -2]","[-3, 0, -2, 1, -3]","[1, 2, -6, -6, -6, 8, 8, 12, -14, -14, 15]",2,11
99263,"[-2, 2, -2, 2, -3]","[-3, 0, -1, 0, -2]","[1, 2, -6, -6, -6, -6, -6, 8, 10, 12, 14, -17]",10,12
100371,"[-3, -2, 2, -1, 2]","[-3, 3, 0, 1, -1]","[1, 2, -6, -6, -6, 8, -9, -11, 15, 15, 21, -24]",12,12
114139,"[-2, -1, -2, 3, -1]","[-2, -3, -1, -3, 1]","[1, 2, -6, -6, -6, -8, 13, 14, 16, 18, -19, -19]",9,12
114376,"[-2, 3, -3, 1, -3]","[-2, -3, -1, -2, -1]","[1, 2, -6, -6, -6, -9, 12, 16, 18, 18, -19, -21]",11,12


In [38]:
if True:


18 {1, -19} set() set()
18


In [37]:
drs

[(-19, 1)]

In [85]:
[m for m in majorana if abs(2*m)!=sp]

[]

In [46]:
get_massles([1, 13, -19, -22],-9)

[1, -19]

In [25]:
    #Fix first set of repeated integers as 𝜈
    νs=list(set([x for x in l if l.count(x)>=2 ]))
    ν=νs[0]
    #implement the phenomenological condition:
    s=-ν
    #Remove neutrinos from list, `l` and get extra quiral fermions
    ψs=[x for x in l if x!=ν]
    if len(ψs)<=2:
        ms=ψs
    else:
        ms=set([x for x in itertools.permutations(ψs,2) if x[0]<=x[1] ])
    xs=[ x for x in ms if abs(x[0]+x[1])==abs(s) ]
    #flatten list of list into list
    massive=[x for sublist in xs for x in sublist]

    return list( set(ψs).difference(set(massive)) )

In [ ]:
cond

,l,k,solution,gcd,n,nmax,massless
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5,[]
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6,[]
2,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,11,"[11, 5, -9]"
3,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,11,"[1, -4, -11]"
4,"[-2, -1]","[-2, -3]","[2, 2, 3, -8, -12, 13]",1,6,13,"[-8, 3, -12, 13]"
...,...,...,...,...,...,...,...
280954,"[-2, -3, -1, 3, 2]","[-1, 3, 1, -1, -3]","[1, 2, -10, 12, 19, 19, -23, -24, -30, -30, 32...",2,12,32,"[1, 2, -30, -24, -23, 12, 19, -10]"
280955,"[-1, -2, 0, 3, 2]","[-1, 3, 1, -1, -2]","[1, 1, 1, 1, 8, -12, -19, 20, 21, -23, -31, 32]",1,12,32,"[8, -23, -12, 21]"
280956,"[-1, 2, -3, -2, -3]","[-1, 3, 2, -2, -1]","[1, 2, 2, 4, 5, -7, -7, -9, -11, 25, 27, -32]",2,12,32,"[1, 27]"
280957,"[-1, 3, -2, 2, -2]","[-1, 3, 2, -1, -2]","[1, 3, 4, 8, -9, -9, -12, 19, 19, -28, -28, 32]",1,12,32,"[32, 1, 3, 4, -28, 8, -12, -9]"


In [ ]:
sltns=dsp[dsp['massless'].apply(len)<=2].reset_index(drop=True)

In [ ]:
sltns.shape

(12584, 7)

In [ ]:
sltns[:10]

,l,k,solution,gcd,n,nmax,massless
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5,[]
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6,[]
2,"[1, 0]","[-1, -5, 0]","[2, -3, -10, 13, 13, -15]",2,6,15,[]
3,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,6,17,"[8, -11]"
4,"[1, 2]","[-1, -2, 0]","[8, -11, -11, 16, 17, -19]",1,6,19,"[16, 17]"
5,"[1, 0]","[-1, -4, 0]","[3, -5, -12, 17, 17, -20]",1,6,20,[]
6,"[1, -2]","[-1, -4, 0]","[10, -13, -13, 17, 22, -23]",1,6,23,"[17, 22]"
7,"[1, 0]","[-1, -7, 0]","[3, -4, -21, 25, 25, -28]",2,6,28,[]
8,"[1, 2]","[-1, 2, 0]","[4, -11, -11, 20, 29, -31]",1,6,31,"[4, 29]"
9,"[0, 3]","[-1, 2, 1]","[8, -13, -13, 19, 31, -32]",2,6,32,"[8, 31]"


In [93]:
t=pd.read_html("https://minciencias.gov.co/convocatorias/todas")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [96]:
import helium as hell

In [98]:
browser=hell.start_firefox(headless=True)

In [99]:
browser.get( 'https://minciencias.gov.co/convocatorias/todas'  )

In [101]:
t=pd.read_html( browser.page_source)
t[0]

,Número,Título,Descripción,Total de recursos,Fecha de Apertura
0,908,"Convocatoria “nuevo conocimiento, desarrollo t...","Apoyar proyectos de investigación aplicada, de...",$4.665.000.000,"Miércoles, Junio 30, 2021"
1,907,Convocatoria jóvenes investigadores e innovado...,Conformar un banco de propuestas elegibles que...,NaN,"Viernes, Junio 11, 2021"
2,906,Convocatoria de Doctorados en el Exterior,Apoyar la formación de profesionales colombian...,NaN,"Miércoles, Mayo 19, 2021"
3,905,Convocatoria para el fortalecimiento de Centro...,Fortalecer los centros de investigación autóno...,$17.000.000.000,"Viernes, Abril 30, 2021"
4,904,Convocatoria para el registro de propuestas qu...,Estimular la inversión privada en Actividades ...,NaN,"Viernes, Abril 23, 2021"
